In [ ]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
import imutils

class DocumentCropper:
    def __init__(self):
        pass
    
    def order_points(self, pts):
        """Order points in clockwise order: top-left, top-right, bottom-right, bottom-left"""
        # Sort the points based on their x-coordinates
        xSorted = pts[np.argsort(pts[:, 0]), :]
        
        # Grab the left-most and right-most points from the sorted x-coordinate points
        leftMost = xSorted[:2, :]
        rightMost = xSorted[2:, :]
        
        # Sort the left-most points according to their y-coordinates
        leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
        (tl, bl) = leftMost
        
        # Use the distance to find top-right and bottom-right
        D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
        (br, tr) = rightMost[np.argsort(D)[::-1], :]
        
        return np.array([tl, tr, br, bl], dtype="float32")
    
    def four_point_transform(self, image, pts):
        """Apply perspective transform to get bird's eye view"""
        rect = self.order_points(pts)
        (tl, tr, br, bl) = rect
        
        # Compute the width of the new image
        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))
        
        # Compute the height of the new image
        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
        
        # Destination points for the perspective transform
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype="float32")
        
        # Compute the perspective transform matrix and apply it
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        
        return warped
    
    def find_document_contour(self, image):
        """Find the largest rectangular contour (white document like bank check)"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Apply bilateral filter to reduce noise while keeping edges sharp
        filtered = cv2.bilateralFilter(gray, 11, 17, 17)
        
        # Apply threshold to highlight white/light areas (documents)
        # This helps separate white documents from darker backgrounds
        _, thresh = cv2.threshold(filtered, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Morphological operations to clean up the binary image
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        
        # Edge detection on the thresholded image
        edged = cv2.Canny(thresh, 50, 150)
        
        # Dilate edges to connect broken contours
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        edged = cv2.dilate(edged, kernel, iterations=1)
        
        # Find contours
        contours = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        
        # Look for the largest rectangular contour that's likely a document
        for c in contours:
            area = cv2.contourArea(c)
            
            # Skip very small contours
            if area < 10000:
                continue
                
            # Approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.015 * peri, True)  # More flexible approximation
            
            # Check if it's roughly rectangular (4 points) and large enough
            if len(approx) == 4:
                # Additional check: ensure it's not too thin (aspect ratio check)
                x, y, w, h = cv2.boundingRect(approx)
                aspect_ratio = float(w) / h
                
                # Bank checks typically have aspect ratio between 2:1 and 3:1
                if 1.5 < aspect_ratio < 4.0 and area > 20000:
                    return approx
            
            # If we can't find exact 4 points, try with more flexible approximation
            elif 4 <= len(approx) <= 8:
                # For irregular contours, use bounding rectangle
                rect = cv2.minAreaRect(c)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                
                # Check area and aspect ratio
                width = rect[1][0]
                height = rect[1][1]
                if width > 0 and height > 0:
                    aspect_ratio = max(width, height) / min(width, height)
                    if 1.5 < aspect_ratio < 4.0 and area > 20000:
                        return box.reshape(4, 2).astype(np.float32)
        
        return None
    
    def enhance_image(self, image):
        """Enhance image quality for better edge detection"""
        # Convert to LAB color space
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        
        # Apply CLAHE to L channel
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        l = clahe.apply(l)
        
        # Merge channels and convert back to BGR
        enhanced = cv2.merge([l, a, b])
        enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)
        
        return enhanced
    
    def crop_document(self, image_path, output_path=None, show_steps=False):
        """Main function to crop document from image"""
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not load image from {image_path}")
        
        orig = image.copy()
        
        # Resize image for processing (optional, for large images)
        ratio = image.shape[0] / 500.0
        image = imutils.resize(image, height=500)
        
        # Enhance image quality
        enhanced = self.enhance_image(image)
        
        if show_steps:
            cv2.imshow("Original", image)
            cv2.imshow("Enhanced", enhanced)
            
            # Show the preprocessing steps for debugging
            gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
            filtered = cv2.bilateralFilter(gray, 11, 17, 17)
            _, thresh = cv2.threshold(filtered, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            cv2.imshow("Grayscale", gray)
            cv2.imshow("Filtered", filtered)
            cv2.imshow("Threshold", thresh)
        
        # Find document contour
        document_contour = self.find_document_contour(enhanced)
        
        if document_contour is None:
            print("Could not find document contour. Trying alternative method...")
            # Alternative: Look for the largest white region
            gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
            _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
            
            contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = imutils.grab_contours(contours)
            
            if contours:
                # Get the largest white region
                largest_contour = max(contours, key=cv2.contourArea)
                if cv2.contourArea(largest_contour) > 10000:
                    # Use bounding rectangle of the largest white area
                    rect = cv2.minAreaRect(largest_contour)
                    box = cv2.boxPoints(rect)
                    document_contour = box.reshape(4, 2).astype(np.float32)
                else:
                    # Fallback: use center portion of image
                    h, w = image.shape[:2]
                    margin = min(w, h) // 10
                    document_contour = np.array([
                        [margin, margin], 
                        [w-margin, margin], 
                        [w-margin, h-margin], 
                        [margin, h-margin]
                    ], dtype=np.float32)
            else:
                # Last resort: use most of the image
                h, w = image.shape[:2]
                document_contour = np.array([[0, 0], [w, 0], [w, h], [0, h]], dtype=np.float32)
        
        # Scale the contour back to original image size
        if document_contour is not None:
            document_contour = document_contour.reshape(4, 2) * ratio
        
        if show_steps:
            # Draw the contour on the original image
            display_orig = orig.copy()
            cv2.drawContours(display_orig, [document_contour.astype(int)], -1, (0, 255, 0), 3)
            cv2.imshow("Document Contour", imutils.resize(display_orig, height=500))
        
        # Apply perspective transform
        warped = self.four_point_transform(orig, document_contour.reshape(4, 2))
        
        if show_steps:
            cv2.imshow("Cropped Document", imutils.resize(warped, height=500))
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        # Save the result if output path is provided
        if output_path:
            cv2.imwrite(output_path, warped)
            print(f"Cropped document saved to {output_path}")
        
        return warped
    
    def crop_multiple_documents(self, image_path, output_dir="cropped_docs", min_area=10000):
        """Detect and crop multiple documents from a single image"""
        import os
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Load image
        image = cv2.imread(image_path)
        orig = image.copy()
        
        # Resize for processing
        ratio = image.shape[0] / 500.0
        image = imutils.resize(image, height=500)
        
        # Enhance image
        enhanced = self.enhance_image(image)
        
        # Convert to grayscale and detect edges
        gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edged = cv2.Canny(blurred, 75, 200)
        
        # Find contours
        contours = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        
        doc_count = 0
        cropped_docs = []
        
        for c in contours:
            # Skip small contours
            if cv2.contourArea(c) < min_area / (ratio ** 2):
                continue
            
            # Approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            
            # If contour has 4 points, it might be a document
            if len(approx) == 4:
                # Scale back to original size
                scaled_contour = approx.reshape(4, 2) * ratio
                
                # Apply perspective transform
                try:
                    warped = self.four_point_transform(orig, scaled_contour)
                    
                    # Save the cropped document
                    output_path = os.path.join(output_dir, f"document_{doc_count}.jpg")
                    cv2.imwrite(output_path, warped)
                    cropped_docs.append(output_path)
                    doc_count += 1
                    
                    print(f"Document {doc_count} saved to {output_path}")
                    
                except Exception as e:
                    print(f"Error processing contour: {e}")
                    continue
        
        print(f"Total documents found and cropped: {doc_count}")
        return cropped_docs

# Usage example
if __name__ == "__main__":
    cropper = DocumentCropper()
    
    # Example 1: Crop single document
    try:
        # Replace 'input_image.jpg' with your image path
        cropped = cropper.crop_document('test_image_processing.jpeg', show_steps=True)
        print("Document cropping completed!")
    except Exception as e:
        print(f"Error: {e}")
    
    # Example 2: Crop multiple documents from one image
    try:
        documents = cropper.crop_multiple_documents('input_image.jpg')
        print(f"Cropped {len(documents)} documents")
    except Exception as e:
        print(f"Error: {e}")
    
    # Example 3: Just get the cropped image without saving
    try:
        cropped_image = cropper.crop_document('input_image.jpg')
        # Now you can process cropped_image further
    except Exception as e:
        print(f"Error: {e}")

Could not find document contour. Trying alternative method...


In [1]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
import imutils

class DocumentCropper:
    def __init__(self):
        pass
    
    def order_points(self, pts):
        """Order points in clockwise order: top-left, top-right, bottom-right, bottom-left"""
        # Sort the points based on their x-coordinates
        xSorted = pts[np.argsort(pts[:, 0]), :]
        
        # Grab the left-most and right-most points from the sorted x-coordinate points
        leftMost = xSorted[:2, :]
        rightMost = xSorted[2:, :]
        
        # Sort the left-most points according to their y-coordinates
        leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
        (tl, bl) = leftMost
        
        # Use the distance to find top-right and bottom-right
        D = dist.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
        (br, tr) = rightMost[np.argsort(D)[::-1], :]
        
        return np.array([tl, tr, br, bl], dtype="float32")
    
    def four_point_transform(self, image, pts):
        """Apply perspective transform to get bird's eye view"""
        rect = self.order_points(pts)
        (tl, tr, br, bl) = rect
        
        # Compute the width of the new image
        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))
        
        # Compute the height of the new image
        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
        
        # Destination points for the perspective transform
        dst = np.array([
            [0, 0],
            [maxWidth - 1, 0],
            [maxWidth - 1, maxHeight - 1],
            [0, maxHeight - 1]], dtype="float32")
        
        # Compute the perspective transform matrix and apply it
        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        
        return warped
    
    def find_document_contour(self, image):
        """Find document contour using edge detection approach for bank checks"""
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Method 1: Enhanced edge detection approach
        # Apply bilateral filter to smooth while preserving edges
        filtered = cv2.bilateralFilter(gray, 9, 75, 75)
        
        # Adaptive thresholding to handle varying lighting
        adaptive_thresh = cv2.adaptiveThreshold(filtered, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                              cv2.THRESH_BINARY, 11, 2)
        
        # Edge detection with multiple techniques
        # Canny edge detection
        edges1 = cv2.Canny(filtered, 30, 80)
        
        # Gradient-based edge detection
        sobelx = cv2.Sobel(filtered, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(filtered, cv2.CV_64F, 0, 1, ksize=3)
        edges2 = np.sqrt(sobelx**2 + sobely**2).astype(np.uint8)
        edges2 = cv2.threshold(edges2, 50, 255, cv2.THRESH_BINARY)[1]
        
        # Combine edge detection methods
        edges = cv2.bitwise_or(edges1, edges2)
        
        # Morphological operations to connect broken edges and close gaps
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
        edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel, iterations=2)
        edges = cv2.dilate(edges, kernel, iterations=1)
        
        # Find contours
        contours = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        
        if not contours:
            return None
            
        # Sort contours by area
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        
        # Method 2: Look for document-like contours
        for i, c in enumerate(contours[:10]):  # Check top 10 largest contours
            area = cv2.contourArea(c)
            
            # Skip very small contours
            if area < 5000:
                continue
            
            # Get bounding rectangle
            x, y, w, h = cv2.boundingRect(c)
            
            # Check if it looks like a document (reasonable aspect ratio)
            aspect_ratio = float(w) / h if h > 0 else 0
            
            # Bank checks typically have aspect ratio between 1.8 and 3.2
            if 1.5 < aspect_ratio < 4.0:
                # Try to approximate the contour
                peri = cv2.arcLength(c, True)
                
                # Try different approximation levels
                for epsilon in [0.01, 0.015, 0.02, 0.03, 0.04]:
                    approx = cv2.approxPolyDP(c, epsilon * peri, True)
                    
                    if len(approx) == 4:
                        # Check if the approximated contour is reasonable
                        approx_area = cv2.contourArea(approx)
                        if approx_area > area * 0.7:  # Should be at least 70% of original area
                            return approx
                
                # If we can't get exactly 4 points, use minimum area rectangle
                rect = cv2.minAreaRect(c)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                
                # Verify this looks like a document
                rect_area = cv2.contourArea(box)
                if rect_area > area * 0.8 and rect_area > 10000:
                    return box.reshape(4, 2).astype(np.float32)
        
        # Method 3: If no good contour found, try with different parameters
        # More aggressive edge detection
        edges_aggressive = cv2.Canny(gray, 20, 60)
        kernel_large = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        edges_aggressive = cv2.morphologyEx(edges_aggressive, cv2.MORPH_CLOSE, kernel_large, iterations=3)
        
        contours2 = cv2.findContours(edges_aggressive, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours2 = imutils.grab_contours(contours2)
        
        if contours2:
            # Get the largest contour
            largest_contour = max(contours2, key=cv2.contourArea)
            if cv2.contourArea(largest_contour) > 8000:
                # Use minimum area rectangle
                rect = cv2.minAreaRect(largest_contour)
                box = cv2.boxPoints(rect)
                return box.reshape(4, 2).astype(np.float32)
        
        return None
    
    def enhance_image(self, image):
        """Enhance image quality for better edge detection"""
        # Convert to LAB color space
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        
        # Apply CLAHE to L channel
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        l = clahe.apply(l)
        
        # Merge channels and convert back to BGR
        enhanced = cv2.merge([l, a, b])
        enhanced = cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)
        
        return enhanced
    
    def crop_document(self, image_path, output_path=None, show_steps=False):
        """Main function to crop document from image"""
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not load image from {image_path}")
        
        orig = image.copy()
        
        # Resize image for processing (optional, for large images)
        ratio = image.shape[0] / 500.0
        image = imutils.resize(image, height=500)
        
        # Enhance image quality
        enhanced = self.enhance_image(image)
        
        if show_steps:
            cv2.imshow("Original", image)
            cv2.imshow("Enhanced", enhanced)
            
            # Show the preprocessing steps for debugging
            gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
            filtered = cv2.bilateralFilter(gray, 9, 75, 75)
            
            # Show edge detection steps
            edges1 = cv2.Canny(filtered, 30, 80)
            sobelx = cv2.Sobel(filtered, cv2.CV_64F, 1, 0, ksize=3)
            sobely = cv2.Sobel(filtered, cv2.CV_64F, 0, 1, ksize=3)
            edges2 = np.sqrt(sobelx**2 + sobely**2).astype(np.uint8)
            edges2 = cv2.threshold(edges2, 50, 255, cv2.THRESH_BINARY)[1]
            combined_edges = cv2.bitwise_or(edges1, edges2)
            
            cv2.imshow("Grayscale", gray)
            cv2.imshow("Filtered", filtered)
            cv2.imshow("Canny Edges", edges1)
            cv2.imshow("Sobel Edges", edges2)
            cv2.imshow("Combined Edges", combined_edges)
        
        # Find document contour
        document_contour = self.find_document_contour(enhanced)
        
        if document_contour is None:
            print("Could not find document contour. Trying alternative method...")
            # Alternative: Look for rectangular shapes in the image using HoughLines
            gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
            
            # Try to detect lines that form a rectangle
            edges = cv2.Canny(gray, 50, 150, apertureSize=3)
            lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)
            
            if lines is not None and len(lines) >= 4:
                # This is a simplified approach - in practice, you'd need to
                # find intersecting lines that form a rectangle
                print("Found some lines, using center crop as fallback")
            
            # Fallback: Use a center crop that's likely to contain the document
            h, w = image.shape[:2]
            margin_w = w // 8  # 12.5% margin on each side
            margin_h = h // 8  # 12.5% margin on top/bottom
            document_contour = np.array([
                [margin_w, margin_h], 
                [w-margin_w, margin_h], 
                [w-margin_w, h-margin_h], 
                [margin_w, h-margin_h]
            ], dtype=np.float32)
        
        # Scale the contour back to original image size
        if document_contour is not None:
            document_contour = document_contour.reshape(4, 2) * ratio
        
        if show_steps:
            # Draw the contour on the original image
            display_orig = orig.copy()
            cv2.drawContours(display_orig, [document_contour.astype(int)], -1, (0, 255, 0), 3)
            cv2.imshow("Document Contour", imutils.resize(display_orig, height=500))
        
        # Apply perspective transform
        warped = self.four_point_transform(orig, document_contour.reshape(4, 2))
        
        if show_steps:
            cv2.imshow("Cropped Document", imutils.resize(warped, height=500))
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        
        # Save the result if output path is provided
        if output_path:
            cv2.imwrite(output_path, warped)
            print(f"Cropped document saved to {output_path}")
        
        return warped
    
    def crop_multiple_documents(self, image_path, output_dir="cropped_docs", min_area=10000):
        """Detect and crop multiple documents from a single image"""
        import os
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Load image
        image = cv2.imread(image_path)
        orig = image.copy()
        
        # Resize for processing
        ratio = image.shape[0] / 500.0
        image = imutils.resize(image, height=500)
        
        # Enhance image
        enhanced = self.enhance_image(image)
        
        # Convert to grayscale and detect edges
        gray = cv2.cvtColor(enhanced, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edged = cv2.Canny(blurred, 75, 200)
        
        # Find contours
        contours = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        contours = imutils.grab_contours(contours)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        
        doc_count = 0
        cropped_docs = []
        
        for c in contours:
            # Skip small contours
            if cv2.contourArea(c) < min_area / (ratio ** 2):
                continue
            
            # Approximate the contour
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            
            # If contour has 4 points, it might be a document
            if len(approx) == 4:
                # Scale back to original size
                scaled_contour = approx.reshape(4, 2) * ratio
                
                # Apply perspective transform
                try:
                    warped = self.four_point_transform(orig, scaled_contour)
                    
                    # Save the cropped document
                    output_path = os.path.join(output_dir, f"document_{doc_count}.jpg")
                    cv2.imwrite(output_path, warped)
                    cropped_docs.append(output_path)
                    doc_count += 1
                    
                    print(f"Document {doc_count} saved to {output_path}")
                    
                except Exception as e:
                    print(f"Error processing contour: {e}")
                    continue
        
        print(f"Total documents found and cropped: {doc_count}")
        return cropped_docs

# Usage example
if __name__ == "__main__":
    cropper = DocumentCropper()
    
    # Example 1: Crop single document
    try:
        # Replace 'input_image.jpg' with your image path
        cropped = cropper.crop_document('test_image_processing.jpeg', show_steps=True)
        print("Document cropping completed!")
    except Exception as e:
        print(f"Error: {e}")
    
    # Example 2: Crop multiple documents from one image
    try:
        documents = cropper.crop_multiple_documents('input_image.jpg')
        print(f"Cropped {len(documents)} documents")
    except Exception as e:
        print(f"Error: {e}")
    
    # Example 3: Just get the cropped image without saving
    try:
        cropped_image = cropper.crop_document('input_image.jpg')
        # Now you can process cropped_image further
    except Exception as e:
        print(f"Error: {e}")

Error: module 'numpy' has no attribute 'int0'
Error: 'NoneType' object has no attribute 'copy'
Error: Could not load image from input_image.jpg


[ WARN:0@0.495] global loadsave.cpp:268 findDecoder imread_('input_image.jpg'): can't open/read file: check file path/integrity
[ WARN:0@0.495] global loadsave.cpp:268 findDecoder imread_('input_image.jpg'): can't open/read file: check file path/integrity
